# **Multivariate Climate Data Clustering using CNN Encoder Model**
Here we are dealing with climate data that comprises spatial information, time information, and scientific values. The dataset contains the value of 7 parameters for a region of 41 longitudes and 41 latitudes for 365 days in a year.

Our goal is to create meaningful clusters of 365 days based on the values of these 7 parameters. As the data dimension is high we planned to use deep learning-based models to generate the latent representation of each day and then generate clusters for 365 days.

We have developed a new CNN Encoder using different convolution neural network layers and activation functions from the Keras library.

# **1. Model Creation:**
This model considers our dataset as an image of size 41x41x7. The model takes a 365x41x41x7 NumPy array as input and applies convolution layers, and max pooling layers to learn the latent features. The output of this model is 512 latent features for each data point.  

In [ ]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout, LeakyReLU
#from keras.preprocessing.image import ImageDataGenerator.  tanh. relu
import numpy as np

def myCNNModel(input_dims):
  '''
  The value of the parameter input_dims is the size of the input image/data we want to train on, for our data it is (41, 41, 7)
  '''
  model = Sequential()
  model.add(Conv2D(input_shape=input_dims,filters=64,kernel_size=(3,3),padding="same", activation="tanh"))
  model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="tanh"))
  model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
  model.add(Dropout(0.1))
  model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="tanh"))
  model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="tanh"))
  model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="tanh"))
  model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
  model.add(Dropout(0.1))
  model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="tanh"))
  model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="tanh"))
  model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="tanh"))
  model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
  model.add(Flatten(name='flatten'))
  model.add(Dense(2048, activation="sigmoid", name='fc1'))
  model.add(Dense(512,  activation="sigmoid", name='fc2'))
  return model

In [ ]:

from time import time
import numpy as np
import keras.backend as K
from tensorflow.keras.layers import Layer, InputSpec
from keras.layers import Dense, Input, Dropout
from keras.models import Model
from keras.optimizers import SGD
from keras import callbacks
from keras.initializers import VarianceScaling
from sklearn.cluster import KMeans


class ClusteringLayer(Layer):

    def __init__(self, n_clusters, weights=None, alpha=1.0, **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(ClusteringLayer, self).__init__(**kwargs)
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.initial_weights = weights
        self.input_spec = InputSpec(ndim=2)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]
        self.input_spec = InputSpec(dtype=K.floatx(), shape=(None, input_dim))
        self.clusters = self.add_weight(shape=(self.n_clusters, input_dim), initializer='glorot_uniform', name='clusters')
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    def call(self, inputs, **kwargs):
        """ student t-distribution, as same as used in t-SNE algorithm.
                 q_ij = 1/(1+dist(x_i, u_j)^2), then normalize it.
        Arguments:
            inputs: the variable containing data, shape=(n_samples, n_features)
        Return:
            q: student's t-distribution, or soft labels for each sample. shape=(n_samples, n_clusters)
        """
        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1))
        return q

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 2
        return input_shape[0], self.n_clusters

    def get_config(self):
        config = {'n_clusters': self.n_clusters}
        base_config = super(ClusteringLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


class CNNModel(object):
    def __init__(self,
                 dims,
                 n_clusters=10,
                 alpha=1.0,
                 init='glorot_uniform'):

        super(CNNModel, self).__init__()

        self.dims = dims
        #self.input_dim = dims[0]
        self.n_stacks = len(self.dims) - 1

        self.n_clusters = n_clusters
        self.alpha = alpha
        self.model_cnn = myCNNModel(self.dims);
        print("====Model created=====")

        # prepare the CNN model with cnn_layers+clustering _layer
        clustering_layer = ClusteringLayer(self.n_clusters, name='clustering')(self.model_cnn.output)
        print("====== clustering layer created ========")
        self.model = Model(inputs=self.model_cnn.input, outputs=clustering_layer)


    def load_weights(self, weights):
        self.model.load_weights(weights)

    def extract_features(self, x):
        return self.model_cnn.predict(x)

    def predict(self, x):  # predict cluster labels using the output of clustering layer
        q = self.model.predict(x, verbose=0)
        return q.argmax(1)

    @staticmethod
    def target_distribution(q):
        weight = q ** 2 / q.sum(0)
        return (weight.T / weight.sum(1)).T

    def compile(self, optimizer='sgd', loss='kld'):
        self.model.compile(optimizer=optimizer, loss=loss)

    def fit(self, x, y=None, maxiter=2e4, batch_size=256, tol=1e-3,
            update_interval=140, save_dir='./results/temp'):

        print('Update interval', update_interval)
        #save_interval = int(x.shape[0] / batch_size) * 5  # 5 epochs
        save_interval = 500
        print('Save interval', save_interval)

        # Step 1: initialize cluster centers using k-means
        t1 = time()
        print('Initializing cluster centers with k-means.')
        kmeans = KMeans(n_clusters=self.n_clusters, n_init=20)
        y_pred = kmeans.fit_predict(self.model_cnn.predict(x))
        y_pred_last = np.copy(y_pred)
        self.model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])

        # Step 2: deep clustering
        # logging file
        import csv
        logfile = open(save_dir + '/cnn_log.csv', 'w')
        logwriter = csv.DictWriter(logfile, fieldnames=['iter','loss'])
        logwriter.writeheader()

        loss = 0
        index = 0
        index_array = np.arange(x.shape[0])
        for ite in range(int(maxiter)):
            if ite % update_interval == 0:
                q = self.model.predict(x, verbose=0)
                p = self.target_distribution(q)  # update the auxiliary target distribution p

                # evaluate the clustering performance
                y_pred = q.argmax(1)
                print("#### inside iteration ### ", ite)

                # check stop criterion
                delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
                print("##### Prediction in side the iter and the delta_label is ", delta_label, ", cluster change= ", np.sum(y_pred != y_pred_last))
                y_pred_last = np.copy(y_pred)
                if ite > 0 and delta_label < tol:
                    print('delta_label ', delta_label, '< tol ', tol)
                    print('Reached tolerance threshold. Stopping training.')
                    logfile.close()
                    break

            # train on batch
            # if index == 0:
            #     np.random.shuffle(index_array)
            idx = index_array[index * batch_size: min((index+1) * batch_size, x.shape[0])]
            loss = self.model.train_on_batch(x=x[idx], y=p[idx])
            print("#### the loss is ", loss)
            index = index + 1 if (index + 1) * batch_size <= x.shape[0] else 0

            # save intermediate model
            if ite % save_interval == 0:
                print('saving model to:', save_dir + '/CNN_model_' + str(ite) + '.h5')
                self.model.save_weights(save_dir + '/CNN_model_' + str(ite) + '.h5')

            ite += 1

        # save the trained model
        logfile.close()
        file_name  = "/CNN_model_final_" + str(round(time()))+ ".h5"
        print('saving model to:', save_dir + file_name)
        self.model.save_weights(save_dir + file_name)

        return y_pred

In [ ]:
! pip install netCDF4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.0 MB/s eta 0:00:00


In [ ]:
import netCDF4 as nc
import pandas as pd
import numpy as np
import xarray as xr

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **2. Data preparation**
The dataset has some NaN values in the SST variable. To replace these NaN values we used the mean value of the full dataset. The function returns 2 NumPy arrays one with size (365, 11767) and another with size (365, 41, 41, 7). The array with size (365, 11767) is used to calculate the silhouette score and the array with size (365, 41, 41, 7) is used to train the model.

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

def data_preprocessing(data_path):
  rdata_daily = xr.open_dataset(data_path)    # data_path = '/content/drive/MyDrive/ERA5_Dataset.nc'
  rdata_daily_np_array = np.array(rdata_daily.to_array())   # the shape of the dailt data is (7, 365, 41, 41)
  rdata_daily_np_array_T = rdata_daily_np_array.transpose(1,0,2,3)   # transform the dailt data from (7, 365, 41, 41) to (365, 7, 41, 41)
  overall_mean = np.nanmean(rdata_daily_np_array_T[:, :, :, :])
  for i in range(rdata_daily_np_array_T.shape[0]):
    for j in range(rdata_daily_np_array_T.shape[1]):
      for k in range(rdata_daily_np_array_T.shape[2]):
        for l in range(rdata_daily_np_array_T.shape[3]):
          if np.isnan(rdata_daily_np_array_T[i, j, k, l]):
            #print("NAN data in ", i, j, k, l)
            rdata_daily_np_array_T[i, j, k, l] = overall_mean
  rdata_daily_np_array_T = rdata_daily_np_array_T.transpose(0,2,3,1)
  rdata_daily_np_array_T_R = rdata_daily_np_array_T.reshape((rdata_daily_np_array_T.shape[0], -1))  # transform the dailt data from (365, 7, 41, 41) to (365, 11767)
  min_max_scaler = preprocessing.MinMaxScaler() # calling the function
  rdata_daily_np_array_T_R_nor = min_max_scaler.fit_transform(rdata_daily_np_array_T_R)   # now normalize the data, otherwise the loss will be very big
  #rdata_daily_np_array_T_R_nor = np.float32(rdata_daily_np_array_T_R_nor)    # convert the data type to float32, otherwise the loass will be out-of-limit
  rdata_daily_np_array_T_R_nor_R = rdata_daily_np_array_T_R_nor.reshape((rdata_daily_np_array_T_R_nor.shape[0], rdata_daily_np_array.shape[2], rdata_daily_np_array.shape[3], rdata_daily_np_array.shape[0]))
  return rdata_daily_np_array_T_R_nor, rdata_daily_np_array_T_R_nor_R

In [ ]:
data_nor_eval, data_clustering = data_preprocessing('/content/drive/MyDrive/ERA5_Dataset.nc')

In [ ]:
data_nor_eval.shape, data_clustering.shape

((365, 11767), (365, 41, 41, 7))

This function calculates the silhoutte score of the formed clusters.

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score
def silhouette_score(X, labels, *, metric="cosine", sample_size=None, random_state=None, **kwds):
 return np.mean(silhouette_samples(X, labels, metric=metric, **kwds))

# **3. Model Training**
This function defines related parameters to train the model. Then instantiate the model and train on the pre-processed data. The model tries to optimize the clustering loss. After training the model returns the cluster assignment for each data point of the input dataset.

In [ ]:
def main():
    # setting the hyper parameters

    batch_size = 8       # Batch size for training the CNN model
    maxiter = 2e4        # Maximum number of iteration the model will train
    update_interval = 30 # After this number of iteration the model will update the cluster assignment
    tol = 0.0000001      # If there is a cluster change more than this tollerance the model training will run
    save_dir = '/content/drive/MyDrive/my_CNN_result'    # The trained model will be stored here

    # load dataset
    x = data_clustering   # Input dataset of the transformed daily data
    y = None              # The cluster level of input data. Not available for our dataset.
    n_clusters = 7        # Number of clusters we want to generate.

    #init = 'glorot_uniform'
    init = VarianceScaling(scale=1. / 3., mode='fan_in', distribution='uniform')

    # prepare the model
    cnnmodel = CNNModel(dims=x.shape[1:4], n_clusters=n_clusters, init=init)

    cnnmodel.model.summary()
    t0 = time()
    cnnmodel.compile(optimizer=SGD(0.0000001, 0.9), loss='kld')
    y_pred = cnnmodel.fit(x, y=y, tol=tol, maxiter=maxiter, batch_size=batch_size,
                     update_interval=update_interval, save_dir=save_dir)
    #print('acc:', metrics.acc(y, y_pred))
    print('clustering time: ', (time() - t0))
    return y_pred

In [ ]:
result = main()
silhouette_avg_rdata_daily = silhouette_score(data_nor_eval, result)
print("The average silhouette_score is :", silhouette_avg_rdata_daily)

In [ ]:
silhouette_avg_rdata_daily = silhouette_score(data_nor_eval, result)
print("The average silhouette_score is :", silhouette_avg_rdata_daily)

The average silhouette_score is : 0.3560080152580256


In [ ]:
from sklearn.metrics import davies_bouldin_score

print("Davies-Bouldin score is ", davies_bouldin_score(data_nor_eval, result))

Davies-Bouldin score is  1.4769867493186457


# **4. Model testing with pre-trained weights:**
To test the model on other datasets we have to create the model and initialize the model with pre-trained weights stored in the drive. Then we have to call the main function to get the clustering results.

In [ ]:
def main_test():
    # setting the hyper parameters

    batch_size = 8       # Batch size for training the CNN model
    maxiter = 2e4        # Maximum number of iteration the model will train
    update_interval = 30 # After this number of iteration the model will update the cluster assignment
    tol = 0.0000001      # If there is a cluster change more than this tollerance the model training will run
    save_dir = '/content/drive/MyDrive/my_CNN_result'    # The trained model will be stored here

    # load dataset
    x = data_clustering   # Input dataset of the transformed daily data
    y = None              # The cluster level of input data. Not available for our dataset.
    n_clusters = 7        # Number of clusters we want to generate.

    #init = 'glorot_uniform'
    init = VarianceScaling(scale=1. / 3., mode='fan_in', distribution='uniform')

    # prepare the model
    cnnmodel = CNNModel(dims=x.shape[1:4], n_clusters=n_clusters, init=init)

    cnnmodel.model.summary()
    cnnmodel.load_weights('/content/drive/MyDrive/my_CNN_result/CNN_model_final.h5')
    t0 = time()
    y_pred = cnnmodel.predict(x)
    print('clustering time: ', (time() - t0))
    return y_pred

In [ ]:
val_res = main_test()
val_res

# **5. Plotting the clustering results:**
To plot the clustering results first we have to create the model and initialize the model with pre-trained weights. Then we will create a new model by taking the Dense layer output and clustering output from the original model. The Dense layer output will be used to plot the clusters using the dimension reduction algorithm.

In [ ]:
n_clusters = 7
init = VarianceScaling(scale=1. / 3., mode='fan_in', distribution='uniform')
cnnmodel = CNNModel(dims=data_clustering.shape[1:5], n_clusters=n_clusters, init=init)

cnnmodel.model.summary()
cnnmodel.load_weights('/content/drive/MyDrive/my_CNN_result/CNN_model_final.h5')

In [ ]:
gradModel = Model(
			inputs=[cnnmodel.model.input],
			outputs=[cnnmodel.model.get_layer('fc2').output,
				cnnmodel.model.output])

In [ ]:
(convOutputs, predictions) = gradModel(data_clustering)

In [ ]:
from matplotlib.lines import Line2D
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

tsne = TSNE(n_components=2, learning_rate='auto', perplexity=10)
tsne_data = tsne.fit_transform(convOutputs)

tsne_df = pd.DataFrame(tsne_data, columns=['TSNE1','TSNE2'])
tsne_df['cluster'] = pd.Categorical(result)

Clusters = {}
Cluster_Centers = {}
for i in set(result):
    Clusters['Cluster' + str(i)] = np.array(tsne_df[tsne_df.cluster == i].drop(columns=['cluster']))
for i in range(len(Clusters)):
    Cluster_Centers[i] = np.mean(Clusters['Cluster' + str(i)],axis=0)

cen_x = [Cluster_Centers[i][0] for i in range(7)]
cen_y = [Cluster_Centers[i][1] for i in range(7)]


tsne_df['cen_x'] = tsne_df.cluster.map({0:Cluster_Centers[0][0], 1:Cluster_Centers[1][0], 2:Cluster_Centers[2][0],
                                        3:Cluster_Centers[3][0], 4:Cluster_Centers[4][0], 5:Cluster_Centers[5][0],
                                        6:Cluster_Centers[6][0]})
tsne_df['cen_y'] = tsne_df.cluster.map({0:Cluster_Centers[0][1], 1:Cluster_Centers[1][1], 2:Cluster_Centers[2][1],
                                        3:Cluster_Centers[3][1], 4:Cluster_Centers[4][1], 5:Cluster_Centers[5][1],
                                        6:Cluster_Centers[6][1]})

colors = ['blue', 'orange', 'green', 'red', 'purple', 'cyan', 'olive']
tsne_df['c'] = tsne_df.cluster.map({0:colors[0], 1:colors[1], 2:colors[2], 3:colors[3], 4:colors[4], 5:colors[5], 6:colors[6]})

fig, ax = plt.subplots(1, figsize=(8,8))
# plot data
plt.scatter(tsne_df.TSNE1, tsne_df.TSNE2, c=tsne_df.c, alpha = 0.6, s=10,)
# plot centroids
plt.scatter(cen_x, cen_y, marker='^', c=colors, s=70)
# plot lines
for idx, val in tsne_df.iterrows():
    x = [val.TSNE1, val.cen_x,]
    y = [val.TSNE2, val.cen_y]
    plt.plot(x, y, c=val.c, alpha=0.2)
# legend
legend_elements = [Line2D([0], [0], marker='o', color='w', label='Cluster {}'.format(i+1),
                   markerfacecolor=mcolor, markersize=5) for i, mcolor in enumerate(colors)]
legend_elements.extend([Line2D([0], [0], marker='^', color='w', label='Centroid - C{}'.format(i+1),
            markerfacecolor=mcolor, markersize=10) for i, mcolor in enumerate(colors)])

plt.legend(handles=legend_elements, title='Clusters', bbox_to_anchor=(1.02, 1), loc='upper left', ncol=2, borderaxespad=0)

plt.title('CNN Encoder Clustering Result\n', loc='left', fontsize=22)
plt.xlabel('Feature-1')
plt.ylabel('Feature-2')

# **6. Evaluation:**
To compute the RMSE, variance, and average inter cluster distance we have to use the xarray format of our real data and the clustering result.

In [ ]:
def total_rmse(data_path,formed_clusters):
  processed_data = data_preprocessing(data_path)
  trans_data = pd.DataFrame(processed_data)
  trans_data['Cluster'] = formed_clusters

  # Normalized
  # Function that creates two dictionaries that hold all the clusters and cluster centers
  def nor_get_clusters_and_centers(input,formed_clusters):
    Clusters = {}
    Cluster_Centers = {}
    for i in set(formed_clusters):
      Clusters['Cluster' + str(i)] = np.array(input[input.Cluster == i].drop(columns=['Cluster']))
      Cluster_Centers['Cluster_Center' + str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
    return Clusters,Cluster_Centers

  intra_rmse = []
  sq_diff = []
  Clusters,Cluster_Centers = nor_get_clusters_and_centers(trans_data,formed_clusters)
  for i in range(len(Clusters)):
    for j in range(len(Clusters['Cluster' + str(i)])):
      diff = Clusters['Cluster' + str(i)][j] - Cluster_Centers['Cluster_Center' + str(i)]
      Sq_diff = (diff)**2
      sq_diff.append(Sq_diff)

  Sq_diff_sum = np.sum(np.sum(sq_diff))
  rmse = np.sqrt(Sq_diff_sum/data_nor_eval.shape[0])
  return rmse

In [ ]:
total_rmse('/content/data/ERA5_Dataset.nc', result)

13.74329829019433

### This cell measure the variances of the generated clusters.  

In [ ]:
trans_data = pd.DataFrame(data_nor_eval)
trans_data['Cluster'] = result
Clusters = {}
Cluster_Centers = {}
for i in set(result):
  Clusters['Cluster' + str(i)] = np.array(trans_data[trans_data.Cluster == i].drop(columns=['Cluster']))

variances = pd.DataFrame(columns=range(len(Clusters)),index=range(2))
for i in range(len(Clusters)):
    variances[i].iloc[0] = np.var(Clusters['Cluster' + str(i)])
    variances[i].iloc[1] = Clusters['Cluster' + str(i)].shape[0]

var_sum = 0
for i in range(7):
    var_sum = var_sum + (variances[i].iloc[0] * variances[i].iloc[1])

var_avg = var_sum/data_nor_eval.shape[0]
var_avg

0.044965697050399205

### The following cell measure the average inter cluster distance.  

In [ ]:
from scipy.spatial.distance import cdist,pdist

trans_data = pd.DataFrame(data_nor_eval)
trans_data['Cluster'] = result
Clusters = {}
Cluster_Centers = {}
for i in set(result):
  Clusters['Cluster' + str(i)] = np.array(trans_data[trans_data.Cluster == i].drop(columns=['Cluster']))

distance_matrix = pd.DataFrame(columns=range(len(Clusters)),index=range(len(Clusters)))
for i in range(len(Clusters)):
  for j in range(len(Clusters)):
    if i == j:
      #distance_matrix[i].iloc[j] = 0
      distance_intra = cdist(Clusters['Cluster' + str(i)], Clusters['Cluster' + str(j)], 'euclidean')
      distance_matrix[i].iloc[j] = np.max(distance_intra)
    elif i > j:
       continue
    else:
      distance = cdist(Clusters['Cluster' + str(i)], Clusters['Cluster' + str(j)], 'euclidean')
      distance_matrix[i].iloc[j] = np.min(distance)
      distance_matrix[j].iloc[i] = np.min(distance)

sum_min = 0
for i in range(n_clusters):
    sum_min = sum_min + np.min(distance_matrix[i])

avg_inter = sum_min/n_clusters
avg_inter

7.313395563952699